In [1]:
%load_ext autoreload
%autoreload 2
import feature_extract
import torch
feature_extract.init_group()

In [4]:
# model = torch.hub.load('facebookresearch/dino:main', 'dino_vits8')
model_string = 'dinov2_vits14'
model = torch.hub.load('facebookresearch/dinov2', model_string)
# data_path = '/global/scratch/tlundy/NFT_Research/nft_research/Dino/images'
# out_path = f'/global/scratch/tlundy/NFT_Research/nft_research/Dino/features/{model_string}'
data_path = '/global/scratch/tlundy/NFT_Research/nft_research/Dino/expanded_images'
out_path = f'/global/scratch/tlundy/NFT_Research/nft_research/Dino/expanded_features/{model_string}'
# out_path = f'/global/scratch/tlundy/NFT_Research/nft_research/Dino/features/test'


Using cache found in /ubc/cs/home/t/tlundy/.cache/torch/hub/facebookresearch_dinov2_main


In [5]:
features = feature_extract.extract_feature_pipeline(model,data_path,out_path,use_cuda=True)

Data loaded with 373074 val imgs.
Extracting features for val set...
Storing features into tensor of shape torch.Size([373074, 384])
  [   0/2915]  eta: 0:59:47    time: 1.230683  data: 0.694733  max mem: 1549
  [   1/2915]  eta: 0:37:16    time: 0.767560  data: 0.347531  max mem: 2095
  [   2/2915]  eta: 0:29:44    time: 0.612719  data: 0.231973  max mem: 2095
  [   3/2915]  eta: 0:26:01    time: 0.536285  data: 0.174129  max mem: 2095
  [   4/2915]  eta: 0:24:06    time: 0.497039  data: 0.139513  max mem: 2095
  [   5/2915]  eta: 0:22:35    time: 0.465669  data: 0.116392  max mem: 2095
  [   6/2915]  eta: 0:21:27    time: 0.442711  data: 0.099835  max mem: 2095
  [   7/2915]  eta: 0:20:37    time: 0.425446  data: 0.087426  max mem: 2095
  [   8/2915]  eta: 0:19:58    time: 0.412173  data: 0.077775  max mem: 2095
  [   9/2915]  eta: 0:19:27    time: 0.401659  data: 0.070044  max mem: 2095
  [  10/2915]  eta: 0:19:01    time: 0.392896  data: 0.063726  max mem: 2095
  [  11/2915]  eta: 

In [9]:
feature_path = out_path+'/testfeat.pth'
features = torch.load(feature_path)
labels = feature_extract.get_labels(data_path)
file_names = feature_extract.get_filenames(data_path)
print(file_names[0])
print(labels)

Dataset ReturnIndexDataset
    Number of datapoints: 846348
    Root location: /global/scratch/tlundy/NFT_Research/nft_research/Dino/images/val
('0-project', '100')
tensor([   0,    0,    0,  ..., 3430, 3430, 3430])


In [4]:
print(len(features))
print(len(labels))

846348
846348


In [ ]:
import pandas as pd
features_list = features.tolist()

# Create a pandas DataFrame
data = {'Label': labels.tolist(), 'Features': features_list,'Collection':[x[0] for x in file_names],'NFT_num':[x[1] for x in file_names]}
df = pd.DataFrame(data)
df

In [ ]:
from scipy.spatial import distance
import itertools
from tqdm import tqdm

# Assuming df is your pandas DataFrame with 'Label' and 'Features' columns

# Define a function to compute Euclidean distance between two feature vectors
def euclidean_distance(feature1, feature2):
    return distance.euclidean(feature1, feature2)

# Group the DataFrame by 'Label'
grouped = df.groupby('Label')

# Compute Euclidean distances for all pairs within each group
results = []
for label, group in tqdm(grouped):
    feature_combinations = list(itertools.combinations(group['Features'], 2))
    distances = [euclidean_distance(pair[0], pair[1]) for pair in feature_combinations]
    if distances:
        results.append((label,sum(distances)/len(distances)))
    else:
        print(label,group)

# Print the computed distances
# print(results)




In [ ]:
# Create a DataFrame for average distances
average_distances_df = pd.DataFrame(results, columns=['Label', 'AverageDistance'])

# Merge the original DataFrame with average distances DataFrame on 'Label'
merged_df = pd.merge(df, average_distances_df, on='Label')
merged_df.to_pickle(f'pairwise_full_{model_string}.pkl')

In [8]:

merged_df = df

In [ ]:
# Find the row with the minimum average distance
q_df = merged_df.query('AverageDistance>0')
min_distance_row = merged_df.loc[q_df['AverageDistance'].idxmin()]

# Print the row with the minimum average distance
print(min_distance_row)

In [9]:
import os
from tqdm import tqdm

# Directory path where the files are located
directory_path = '/global/scratch/tlundy/NFT_Research/nft_research/Dino/images/val'

# Create a dictionary to store label-file associations
label_file_dict = {}

# Iterate through the files in the directory
for idx,filename in tqdm(enumerate(sorted(os.listdir(directory_path)))):
    # Extract the label from the file name (assuming the label is in the file name)
    # print(idx,filename)
    # Check if the label exists in the merged_df DataFrame
    if idx in merged_df['Label'].values:
        # If yes, associate the file name with the label
        label_file_dict[idx] = filename

3431it [00:00, 3867.80it/s]


In [10]:
merged_df['FileName'] = merged_df['Label'].map(label_file_dict)

# Print the updated DataFrame
merged_df


,Label,Features,FileName
0,0,"[-0.06209539249539375, 0.007882727310061455, 0...",0-project
1,0,"[-0.024552905932068825, -0.003494466654956341,...",0-project
2,0,"[-0.045510295778512955, 0.0013163626426830888,...",0-project
3,0,"[-0.011078260838985443, -0.001131890807300806,...",0-project
4,0,"[0.007828008383512497, 0.003978794906288385, -...",0-project
...,...,...,...
846343,3430,"[0.011794660240411758, 0.007974197156727314, 0...",zuckbots
846344,3430,"[-0.02035275660455227, -0.0059893010184168816,...",zuckbots
846345,3430,"[0.022820619866251945, -0.04797149449586868, 0...",zuckbots
846346,3430,"[-0.013995502144098282, -0.006069438066333532,...",zuckbots


In [11]:
import numpy as np
# Group the DataFrame by 'Label'
grouped = merged_df.groupby('Label')

# Compute average feature vector for each label
average_feature_vectors = []
for label, group in tqdm(grouped):
    # Compute average feature vector for the current label
    avg_feature_vector = np.mean(group['Features'].tolist(), axis=0)
    average_feature_vectors.append((label, avg_feature_vector))
average_features_df = pd.DataFrame(average_feature_vectors, columns=['Label', 'AverageFeatureVector'])
merged_df2 = pd.merge(merged_df, average_features_df, on='Label')

100%|██████████████████████████████████████████████████████████████████████████████████████████| 3431/3431 [04:41<00:00, 12.18it/s]


In [12]:
merged_df2

,Label,Features,FileName,AverageFeatureVector
0,0,"[-0.06209539249539375, 0.007882727310061455, 0...",0-project,"[-0.031895837579454474, 0.003036754775425623, ..."
1,0,"[-0.024552905932068825, -0.003494466654956341,...",0-project,"[-0.031895837579454474, 0.003036754775425623, ..."
2,0,"[-0.045510295778512955, 0.0013163626426830888,...",0-project,"[-0.031895837579454474, 0.003036754775425623, ..."
3,0,"[-0.011078260838985443, -0.001131890807300806,...",0-project,"[-0.031895837579454474, 0.003036754775425623, ..."
4,0,"[0.007828008383512497, 0.003978794906288385, -...",0-project,"[-0.031895837579454474, 0.003036754775425623, ..."
...,...,...,...,...
846343,3430,"[0.011794660240411758, 0.007974197156727314, 0...",zuckbots,"[0.003040199307917507, -0.03182816090190439, 0..."
846344,3430,"[-0.02035275660455227, -0.0059893010184168816,...",zuckbots,"[0.003040199307917507, -0.03182816090190439, 0..."
846345,3430,"[0.022820619866251945, -0.04797149449586868, 0...",zuckbots,"[0.003040199307917507, -0.03182816090190439, 0..."
846346,3430,"[-0.013995502144098282, -0.006069438066333532,...",zuckbots,"[0.003040199307917507, -0.03182816090190439, 0..."


In [13]:
merged_df2.to_pickle(f'collection_features_{model_string}.pkl')

In [14]:
import itertools
from scipy.spatial import distance
from tqdm import tqdm
# Assuming merged_df is your pandas DataFrame with 'Label' and 'AverageFeatureVector' columns

# Define a function to compute Euclidean distance between two feature vectors
def euclidean_distance(feature1, feature2):
    return distance.euclidean(feature1, feature2)

def dot_distance(feature1,feature2):
    return np.dot(feature1, feature2)
# Create combinations of labels
label_combinations = list(itertools.combinations(merged_df2['FileName'].unique(), 2))
label_to_vec = merged_df2.set_index('FileName')['AverageFeatureVector'].to_dict()
# Compute distances between average feature vectors for every pair of labels
print('Done')
pairwise_distances = []
for label1, label2 in tqdm(label_combinations):
    feature_vector1 = label_to_vec[label1]/np.linalg.norm(label_to_vec[label1])
    feature_vector2 = label_to_vec[label2]/np.linalg.norm(label_to_vec[label2])
    # print(feature_vector1)
    dist1 = euclidean_distance(feature_vector1, feature_vector2)
    dist2 = dot_distance(feature_vector1, feature_vector2)
    pairwise_distances.append((label1, label2, dist1,dist2))

# Create a DataFrame for pairwise distances
pairwise_distances_df = pd.DataFrame(pairwise_distances, columns=['Label1', 'Label2', 'Euc_Distance','Dot_Distance'])

# Print the DataFrame with pairwise distances
pairwise_distances_df


Done


100%|█████████████████████████████████████████████████████████████████████████████████| 5884165/5884165 [02:19<00:00, 42093.35it/s]


,Label1,Label2,Euc_Distance,Dot_Distance
0,0-project,0010-editions,1.185466,0.297335
1,0-project,0n1-comics,1.277081,0.184532
2,0-project,0n1-force,1.257016,0.209955
3,0-project,0n1-frame,1.336737,0.106567
4,0-project,0xai-genesis,1.350424,0.088178
...,...,...,...,...
5884160,zerion-dna-onepointo,zoofrenz-apefrenz-2-0,1.195863,0.284956
5884161,zerion-dna-onepointo,zuckbots,1.183998,0.299075
5884162,zombieclub-token,zoofrenz-apefrenz-2-0,1.032675,0.466791
5884163,zombieclub-token,zuckbots,1.135824,0.354952


In [15]:
del merged_df2

In [16]:
pairwise_distances_df.to_pickle(f'collection_dist_{model_string}.pkl')

In [17]:
# Find the row with the minimum average distance
min_distance_row = pairwise_distances_df.loc[pairwise_distances_df['Dot_Distance'].idxmax()]

# Print the row with the minimum average distance
print(min_distance_row)

Label1          pepe-open-editions-by-matt-furie-deprecated
Label2                                        testnft020202
Euc_Distance                                            0.0
Dot_Distance                                            1.0
Name: 5411869, dtype: object


In [18]:
sorted_df = pairwise_distances_df.sort_values(by='Dot_Distance')

# Get the top 50 rows with the lowest distances
top_50_lowest_rows = sorted_df.tail(50)

# Print the top 50 rows
print(top_50_lowest_rows)

                                               Label1  \
2378456                                 crypto-phunks   
5157641                                     nakamigos   
993539                                          azuki   
5098581                                mr-checkspeare   
2377911                                 crypto-phunks   
708279                                      ape2023-1   
506608                          alien-frens-evolution   
5754435                            snoop-dogg-doggies   
2480403                                   cryptopunks   
2480948                                   cryptopunks   
55046                                          0xzuki   
23765                                  0xapes-trilogy   
241047                  888-inner-circle-orange-realm   
3560033                               food-for-degens   
4563984                                   kryptothugs   
5202641                 nft-das-magazin-by-mike-hager   
1537115                        

In [ ]:
!pip install community

In [ ]:
import networkx as nx
G = nx.from_pandas_edgelist(pairwise_distances_df, 'Label1', 'Label2', ['Dot_Distance','Euc_Distance'], create_using=nx.Graph())
# Find communities using Louvain method
partition = nx.community.louvain_communities(G, weight='Dot_Distance',resolution=1.5)

# Add community information to the graph nodes
nx.set_node_attributes(G, partition, 'community')

In [ ]:
len(partition)

In [ ]:
set_milady = [x for x in partition if 'milady' in x]

In [ ]:
set_milady